In [7]:
import os
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain_community.document_loaders import Docx2txtLoader
from langchain_community.document_loaders import UnstructuredWordDocumentLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS


# 向量库：FAISS

def build_vector_db(knowledge_dir: str, output_dir: str = "./vector_db", model_name: str = "./bge-small-zh-v1.5"):
    """
    从文档目录生成 FAISS 向量库，保存为离线文件。
    :param knowledge_dir: 文档文件夹（PDF/DOC/TXT）
    :param output_dir: 保存路径（FAISS 索引和元数据）
    :param model_name: 嵌入模型（中文优化）
    """
    # 加载文档
    documents = []
    for file in os.listdir(knowledge_dir):
        file_path = os.path.join(knowledge_dir, file)
        ext = os.path.splitext(file)[1].lower()
        try:
            if ext == ".pdf":
                loader = PyPDFLoader(file_path)
            elif ext in [".doc", ".docx"]:
                loader = UnstructuredWordDocumentLoader(file_path)
            elif ext == ".txt":
                loader = TextLoader(file_path, encoding="utf-8")
            else:
                continue
            documents.extend(loader.load())
        except Exception as e:
            print(f"加载 {file_path} 失败: {e}")
    
    # 分块（语义分割，类似你的 bert_semantic_split）
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=300,  # 你的 chunk_size
        chunk_overlap=50,  # 上下文重叠
        separators=["。", "！", "？", "\n\n"],  # 中文标点分句
        length_function=len
    )
    chunks = text_splitter.split_documents(documents)
    
    # 生成嵌入
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs={"device": "cpu"}  # 改 "cuda" 若有 GPU
    )
    
    # 建 FAISS 索引（cosine 相似度）
    vectorstore = FAISS.from_documents(chunks, embeddings)
    
    # 保存
    os.makedirs(output_dir, exist_ok=True)
    vectorstore.save_local(output_dir)
    print(f"向量库已保存到 {output_dir}/index.faiss 和 index.pkl")
    return vectorstore

# 示例调用（提前一天运行）
vectore_store=build_vector_db("知识文档")

向量库已保存到 ./vector_db/index.faiss 和 index.pkl


In [2]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from sentence_transformers import CrossEncoder
from langchain_community.embeddings import SentenceTransformerEmbeddings
import numpy as np

def load_and_query(query: str, vector_db_dir: str = "./vector_db", model_name: str = "./bge-small-zh-v1.5", cross_model_name: str = "./mmarco-mMiniLMv2-L12-H384-v1", top_k: int = 5):
    """
    加载离线向量库，查询并 rerank。
    :param query: 查询语句
    :param vector_db_dir: FAISS 路径
    :param model_name: 嵌入模型
    :param cross_model_name: rerank 模型
    :param top_k: 返回 top-K 结果
    """
    # 加载 FAISS
    embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs={"device": "cpu"})
    vectorstore = FAISS.load_local(vector_db_dir, embeddings, allow_dangerous_deserialization=True)
    
    # 粗召回（k=20）
    retriever = vectorstore.as_retriever(search_kwargs={"k": 20})
    docs = retriever.invoke(query)
    
    # Rerank
    cross_encoder = CrossEncoder(cross_model_name)
    pairs = [[query, doc.page_content] for doc in docs]
    scores = cross_encoder.predict(pairs)
    
    # 按分数排序，取 top-K
    top_indices = np.argsort(scores)[::-1][:top_k]
    top_docs = [docs[i] for i in top_indices]
    top_scores = scores[top_indices]
    
    return top_docs, top_scores


In [4]:
import requests
import os
from typing import List
def call_deepseek_api(query: str, context: str, api_key: str) -> str:
    """
    调用 DeepSeek API，基于查询和 Top-K chunks 生成答案
    """
    # 构造上下文：将 Top-K chunks 拼接为上下文
    prompt = f"""
    你是一个专业的金融领域助手。基于以下上下文，回答用户的问题。确保只给出对应的答案，让答案最准确、简洁。

    问题: {query}

    上下文:
    {context}

    请提供详细的答案。
    """
    
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    
    payload = {
        "model": "deepseek-chat",
        "messages": [
            {"role": "system", "content": "You are a helpful assistant in the financial domain."},
            {"role": "user", "content": prompt}
        ],
        "max_tokens": 500,
        "temperature": 0
    }
    
    try:
        response = requests.post("https://api.deepseek.com/v1/chat/completions", 
                               json=payload, headers=headers)
        response.raise_for_status()
        result = response.json()
        return result['choices'][0]['message']['content']
    except requests.RequestException as e:
        return f"调用 DeepSeek API 失败: {str(e)}"


In [8]:
# 示例查询（现场运行）
query = "支付并签约（APP）交易中，交易子类txnSubType应填写为何值？"
top_docs, scores = load_and_query(query)
print(f"Top-{len(top_docs)} 结果：")
for i, (doc, score) in enumerate(zip(top_docs, scores)):
    print(f"排名 {i+1} (分数: {score:.3f}): {doc.page_content}")

Top-5 结果：
排名 1 (分数: 3.784): Q/CUP-SC071.2.33—2023
9
序号 数据元名称 数据元标识 数据元格式 数据元取值说明
79：开通交易
94：IC卡脚本通知
95：查询更新加密公钥证书
6. 交易子类 txnSubType N2 依据实际交易类型填写
默认取值：00
7. 产品类型 bizType N6
依据实际业务场景填写(目前仅使用后 4
位，签名 2位默认为 00)
默认取值:000000
具体取值范围：
000201B2C网关支付
000301 认证支付 2.0
000302 评级支付
000401 贷记
000501 代收
000601 账单支付
000801 跨行收单
000901 绑定支付
001001 订购
000202 B2B
8. 前台通知地址 frontUrl ANS1..256
前台返回商户结果时使用，前台类交易需
上送
不支持换行符等不可见字符
9. 后台通知地址 backUrl ANS1..256
后台返回商户结果时使用，如上送，则发
送商户后台交易结果通知，不支持换行符
等不可见字符，如需通过专线通知，需要
在通知地址前面加上前缀：专线的首字母
加竖线 ZX|
10. 接入类型 accessType N1
0：商户直连接入
1：收单机构接入
2：平台商户接入
11. 收单机构代码 acqInsCode AN8..11 已被批准加入银联互联网系统的收单机
构代码
12. 商户类别 merCatCode N4 填写 MCC码，接入类型为收单机构接入
时需上送
13. 商户代码 merId AN15 已被批准加入银联互联网系统的商户代
码
14. 商户名称 merName ANS1..40 接入类型为收单机构接入时需上送
不支持换行符等不可见字符
15. 商户英文名称 merEnName ANS40
接入类型为收单机构接入时需上送
仅支持标准 ASCII码中的可见字符，不支
持换行符等不可见字符
1) 总长度 40位
排名 2 (分数: 2.328): 。
5.4.3 请求报文
序号 域名 变量名 出现要求 备注
40. 版本号 version M 固定填写 5.1.0
41. 编码方式 encoding M 默认取值：UTF-8
42. 证书 ID certId C
填 写 签 名 私 钥 证 书 

In [9]:
context = "\n\n".join([f"Chunk {i+1}: {doc.page_content}" for i, doc in enumerate(top_docs)])
print(context)

Chunk 1: Q/CUP-SC071.2.33—2023
9
序号 数据元名称 数据元标识 数据元格式 数据元取值说明
79：开通交易
94：IC卡脚本通知
95：查询更新加密公钥证书
6. 交易子类 txnSubType N2 依据实际交易类型填写
默认取值：00
7. 产品类型 bizType N6
依据实际业务场景填写(目前仅使用后 4
位，签名 2位默认为 00)
默认取值:000000
具体取值范围：
000201B2C网关支付
000301 认证支付 2.0
000302 评级支付
000401 贷记
000501 代收
000601 账单支付
000801 跨行收单
000901 绑定支付
001001 订购
000202 B2B
8. 前台通知地址 frontUrl ANS1..256
前台返回商户结果时使用，前台类交易需
上送
不支持换行符等不可见字符
9. 后台通知地址 backUrl ANS1..256
后台返回商户结果时使用，如上送，则发
送商户后台交易结果通知，不支持换行符
等不可见字符，如需通过专线通知，需要
在通知地址前面加上前缀：专线的首字母
加竖线 ZX|
10. 接入类型 accessType N1
0：商户直连接入
1：收单机构接入
2：平台商户接入
11. 收单机构代码 acqInsCode AN8..11 已被批准加入银联互联网系统的收单机
构代码
12. 商户类别 merCatCode N4 填写 MCC码，接入类型为收单机构接入
时需上送
13. 商户代码 merId AN15 已被批准加入银联互联网系统的商户代
码
14. 商户名称 merName ANS1..40 接入类型为收单机构接入时需上送
不支持换行符等不可见字符
15. 商户英文名称 merEnName ANS40
接入类型为收单机构接入时需上送
仅支持标准 ASCII码中的可见字符，不支
持换行符等不可见字符
1) 总长度 40位

Chunk 2: 。
5.4.3 请求报文
序号 域名 变量名 出现要求 备注
40. 版本号 version M 固定填写 5.1.0
41. 编码方式 encoding M 默认取值：UTF-8
42. 证书 ID certId C
填 写 签 名 私 钥 证 书 的 Serial
Number，该值可通过银联提供的


In [10]:
# 调用 DeepSeek API 生成答案
deepseek_api_key = "sk-afabddf5ba0c4c96abb3567aa3324605"
answer = call_deepseek_api(query, context, deepseek_api_key)
#print(reranked_chunks)
print("DeepSeek API 生成的答案：")
print(answer)

DeepSeek API 生成的答案：
27


In [11]:
import torch

# 检查CUDA
try:
    print("torch version:", torch.__version__)
    cuda_available = torch.cuda.is_available()
    print("CUDA available:", cuda_available)
    if cuda_available:
        print("torch.cuda version:", torch.version.cuda)
        try:
            print("cuDNN version:", torch.backends.cudnn.version())
        except Exception:
            pass
        n = torch.cuda.device_count()
        print(f"CUDA device count: {n}")
        for i in range(n):
            props = torch.cuda.get_device_properties(i)
            print(f"Device {i}: {props.name}")
            print(f"  Capability: {torch.cuda.get_device_capability(i)}")
            print(f"  Total memory (MB): {props.total_memory // (1024**2)}")
            print(f"  Allocated (MB): {torch.cuda.memory_allocated(i) // (1024**2)}")
            print(f"  Reserved (MB): {torch.cuda.memory_reserved(i) // (1024**2)}")
    else:
        print("未检测到 CUDA 设备。若有 GPU，请检查 CUDA 驱动/环境变量 (CUDA_VISIBLE_DEVICES)。")
except Exception as e:
    print("检查 CUDA 时出错:", e)


torch version: 2.8.0+cpu
CUDA available: False
未检测到 CUDA 设备。若有 GPU，请检查 CUDA 驱动/环境变量 (CUDA_VISIBLE_DEVICES)。


In [ ]:
import requests
import os
from typing import List
def call_deepseek_api(query: str, context: str, api_key: str) -> str:
    """
    调用 DeepSeek API，基于查询和 Top-K chunks 生成答案
    """
    # 构造上下文：将 Top-K chunks 拼接为上下文
    prompt = f"""
    你是一个专业的金融领域助手。基于以下上下文，回答用户的问题。确保只给出对应的答案，让答案最准确、简洁。

    问题: {query}

    上下文:
    {context}

    请提供详细的答案。
    """
    
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    
    payload = {
        "model": "deepseek-chat",
        "messages": [
            {"role": "system", "content": "You are a helpful assistant in the financial domain."},
            {"role": "user", "content": prompt}
        ],
        "max_tokens": 500,
        "temperature": 0
    }
    
    try:
        response = requests.post("https://api.deepseek.com/v1/chat/completions", 
                               json=payload, headers=headers)
        response.raise_for_status()
        result = response.json()
        return result['choices'][0]['message']['content']
    except requests.RequestException as e:
        return f"调用 DeepSeek API 失败: {str(e)}"
